<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Gonzalo Claro
- Nombre de alumno 2: Camilo Meneses


### **Link de repositorio de GitHub:** https://github.com/GonzaloClaro/evaluaciones_mds7202_grupo23

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 7.9 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Universidad /2022-1/Lab programación científica/Lab 6/'

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [4]:
df_retail = pd.read_pickle(path+"online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [87]:
def custom_features(dataframe_in):
    #Calculamos Length
    df_length = dataframe_in.copy()
    df_length = (df_length.groupby("Customer ID")["InvoiceDate"].agg(["first", "last"]).reset_index())
    df_length["Length"] = (df_length["last"] - df_length["first"]).dt.days
    df_length = df_length.loc[:, ["Customer ID", "Length"]]
    df_LRMFP = df_length

    #Calculamos Recency
    df_max = (dataframe_in.copy().groupby("Customer ID").max().reset_index())
    df_max = df_max.rename(columns={"InvoiceDate": "Max"}).loc[:, ["Customer ID", "Max"]]
    df_recency = dataframe_in.merge(df_max, how='left')
    df_recency["Recency"] = (df_recency["Max"] + datetime.timedelta(days = 1) - df_recency["InvoiceDate"]).dt.days
    df_recency = (df_recency.groupby("Customer ID")["Recency"].mean().reset_index().round())
    df_LRMFP = pd.merge(df_LRMFP, df_recency, how='left')

    #Calculamos Frequency
    df_frequency = dataframe_in.copy()
    df_frequency = df_frequency.groupby('Invoice').first().reset_index()
    df_frequency = df_frequency.groupby('Customer ID').size().reset_index()
    df_frequency = df_frequency.rename({0: 'Frequency'},axis='columns')
    df_LRMFP = pd.merge(df_LRMFP, df_frequency, how='left')

    #Calculamos Monetary
    df_monetary = dataframe_in.copy()
    df_monetary['Monetary'] = df_monetary['Quantity']*df_monetary['Price']
    df_monetary = df_monetary.groupby('Customer ID').sum().loc[:,'Monetary'].reset_index()
    df_LRMFP = pd.merge(df_LRMFP, df_monetary, how='left')
    df_LRMFP['Monetary'] = df_LRMFP['Monetary']/df_LRMFP['Frequency']
    
    #Calculamos Periodicity
    df_periodicity = dataframe_in.copy()
    df_periodicity = df_periodicity.groupby('Invoice').first().reset_index() 
    df_periodicity = df_periodicity.sort_values(['Customer ID', 'InvoiceDate'])
    df_periodicity['PrevDate'] = df_periodicity.groupby('Customer ID')['InvoiceDate'].shift()
    df_periodicity['Periodicity'] = (df_periodicity['InvoiceDate'] - df_periodicity['PrevDate']).dt.seconds/(3600*24)
    df_periodicity = df_periodicity.groupby('Customer ID').std().loc[:,['Periodicity']].reset_index()
    df_LRMFP = pd.merge(df_LRMFP, df_periodicity, how='left')
    
    df_LRMFP['Customer ID'] = df_LRMFP['Customer ID'].astype(int)
    df_LRMFP = df_LRMFP.dropna().reset_index(drop=True)

    return df_LRMFP

In [88]:
df_lrmfp = custom_features(df_retail)
display(df_lrmfp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.



,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346,196,69.0,11,33.896364,0.477915
1,12349,181,84.0,3,890.380000,0.053524
2,12356,44,24.0,3,1186.766667,0.510688
3,12358,356,159.0,3,906.336667,0.529839
4,12359,308,167.0,6,427.226667,0.445477
...,...,...,...,...,...,...
2078,18272,97,57.0,3,435.800000,0.021606
2079,18276,336,148.0,5,264.132000,0.414071
2080,18277,97,39.0,4,267.417500,0.078341
2081,18283,275,159.0,6,103.228333,0.492313


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [89]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = (X - X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
        return X

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [90]:
LRMFP = Pipeline([('Preprocessing',FunctionTransformer(custom_features)), 
                ('MinMaxTransform', MinMax()),
                ('T-SNE', TSNE())])


proy_LRMFP = LRMFP.fit_transform(df_retail)
proy_LRMFP              

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



array([[  8.427461 ,  27.529516 ],
       [ -6.5408487,  60.46388  ],
       [ 28.976255 ,  26.663485 ],
       ...,
       [-21.73445  ,  77.627396 ],
       [ 22.34733  , -41.71903  ],
       [ 29.348455 , -34.874203 ]], dtype=float32)

Graficamos las componentes obtenidas en reducción de dimensionalidad

In [91]:
fig = px.scatter(proy_LRMFP, x=0, y=1,)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [95]:
inercia = []
num_clusters = []
for i in range(2, 20):
    kmeans = KMeans(n_clusters = i, random_state = 0)
    kmeans.fit(proy_LRMFP)
    num_clusters.append(i)
    inercia.append(kmeans.inertia_)

inercia = pd.DataFrame(list(zip(num_clusters,inercia)), columns=["numero de clusters", "inercia"])

#graficamos el metodo del codo
px.line(inercia, x="numero de clusters", y="inercia", title="Método del Codo con K-Means")

El método del codo calcula la suma de errores cuadráticos dentro del clúster para diferentes valores de K. Posteriormente se elige el K para la cual la suma de errores cuadráticos comienza a disminuir.

Observamos que la disminución del error cuadrático medio, a partir de los 5 clusters, cae manera pronunciada y posteriormente comienza a decaer con una pendiente más cercana a 0.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [93]:
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(proy_LRMFP)
df_lrmfp['Cluster'] = kmeans.labels_
df_LRMFP= pd.DataFrame(proy_LRMFP)
df_LRMFP['Cluster'] = kmeans.labels_

#se crea una tabla con los promedios para cada uno de los atributos, agrupando estos por el clúster que pertenecen
df_lrmfp = df_lrmfp.groupby('Cluster').mean()
df_lrmfp

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,,
0,16966.062738,310.942966,151.410646,10.167300,395.393532,0.438178
1,13709.535047,200.934579,97.815421,5.453271,444.997175,0.461549
2,14072.755056,330.633708,166.321348,12.588764,454.390868,0.431349
3,16279.605201,127.669031,65.460993,4.796690,332.168706,0.483502
4,15394.992337,158.969349,82.827586,3.298851,361.932596,0.047064


**Respuesta** ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?:

Dados los resultados de clustering y el promedio para las métricas de LRFMP se pueden observar agrupaciones coherentes. A continuación se mencionan ciertas características de cada grupo de clientes, teniendo en cuenta que siempre son con respecto al resto de clusters.


* El **cluster 0** corresponde a clientes muy fieles a Mr. Lepin, con un alto Lenght y Frecuency, sólo superados por el grupo número 2. En cuanto a su gasto (Monetary) se ve que se encuentran justo en la mitad, sin gastar mucho ni poco. 
-El **cluster 1** se podría decir que son clientes promedio, pero que destacan en ser de los grupos de clientes que más gastan en promedio, sólo superados por el cluster 2.
-El **cluster 2** que posee los clientes más fieles con diferencia ya que es el grupo con mayor Length (antiguedad), además de ser los que más gastan en promedio (alto Monetary) y los que visitan la tienda más seguido (Frequency). Los clientes top de Mr. Lepin.
-El **cluster 3** son clientes poco fieles, reflejados en ser los con menor Lenght y la segunda menor Frecuency de todos. Son los clientes más nuevos, los que menos gastan y en periodos más separados de tiempo. Por los que Mr. Lepin debería cautivarlos con algún tipo de campaña personalizada.
-El **cluster 4** corresponde a clientes muy poco fieles, relativamente recientes y con aún menor frecuencia de visitas que el grupo anterior. De todas formas tienen la particularidad de ser los que visitan la tienda más seguido pese a tener poca cantidad de visitas totales. Esto puede ser debido a que buscan algun producto en particular probablemente. Habría que poner ojo con Mr. Lepin a cómo enganchar a estos clientes, aprovechando que visitan la tienda regularmente, para que gasten más y la visiten más veces al día. 


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [94]:
fig = px.scatter(df_LRMFP, x=0, y=1, color= 'Cluster' )
fig.show()

**Respuesta:**

Vemos que los clusters son fácilmente diferenciables, en particular el con mayor distancia entre clusters es el grupo 4, que puede destacar al tener una periodicidad mucho menor que el resto de grupos, disparando su diferencia con el resto.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>